In [1]:
########################################################################################################################
# This script runs different imputation (missingness-handling) methods
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import gc
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
from time import time
from typing import List, Literal, Optional, Union
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
####################################################################################################################
# USER_SPECIFIC SETTING
# FEAT_IN_DIR_PATH: Path of the input directory of the feature datasets
####################################################################################################################
FEAT_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# IMPUTE_LIST: A list of strings specifying the imputation methods. (Default: ['Zero', 'Mean', 'Median'])
# Must be a non-empty sub-list of ['Zero', 'Mean', 'Median'].
########################################################################################################################
IMPUTE_LIST: list[str] = ['Zero', 'Mean', 'Median']

In [ ]:
########################################################################################################################
# Define the granularity and partition of the datasets
########################################################################################################################
granular_list: list[str] = ['Patient', 'Encounter']
partitions: list[str] = ['train', 'test']

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds, and also the granularity levels, partitions, and imputation methods
########################################################################################################################
for conf_idx, (granular, C, D, partition, impute) in enumerate(product(granular_list,
                                                               Cs,
                                                               Ds,
                                                               partitions,
                                                               IMPUTE_LIST), 1):
    log_head: str = f'[{conf_idx}. {granular}-level; C={C}; D={D}; partition={partition}; impute={impute}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the feature dataset created in P04_Winsorizing_Scaling.ipynb
    ####################################################################################################################
    feat_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_{granular}_{partition}_v2.parquet')
    df: pd.DataFrame = pd.read_parquet(feat_in_path)
    print(f'{log_head}Feature dataset loaded with dimension = {df.shape}')
    print('*'*120)
    
    ####################################################################################################################
    # Step 1. Identify the features to be excluded from imputation (indexing columns)
    ####################################################################################################################
    idx_cols: list[str] = [c for c in ['PatientDurableKey', 'EncounterKey', 'EncDate'] if c in df.columns]

    ####################################################################################################################
    # Step 2. Run the imputation method by creating a new copy of df as df_imp
    ####################################################################################################################
    t0 = time()
    df_imp = df.drop(columns=idx_cols)
    for col in df_imp.columns:
        df_imp[f'{col}!NA'] = df_imp[col].isna().astype(int)
    if impute == 'Zero':
        df_imp = df_imp.fillna(0)
    elif impute == 'Mean':
        df_imp = df_imp.astype('float').fillna(df_imp.mean())
    elif impute == 'Median':
        df_imp = df_imp.astype('float').fillna(df_imp.median())
    assert df_imp.isna().sum().sum() == 0
    assert df_imp.shape[1] == 2 * (df.shape[1] - len(idx_cols))
    for c in idx_cols[::-1]:
        df_imp.insert(0, c, df[c])
    t1 = time()
    print(f'{log_head}Elapsed time of imputation = {t1-t0:.2f} seconds.')
    
    ####################################################################################################################
    # Save the datasets
    ####################################################################################################################
    feat_out_path: str = feat_in_path.replace('v2.parquet', 'v3.parquet')
    df_imp.to_parquet(feat_out_path)
    print(f'{log_head}Imputed dataset saved as {feat_out_path}')
    print(f'{log_head}Dimension = {df_imp.shape}')
    
    ####################################################################################################################
    # Clear cache
    ####################################################################################################################
    del df
    del df_imp
    gc.collect()
    print(f'-'*120)